# <font color=green>APR - Travaux Pratiques n°8.</font>

> **Ce sujet est en lien avec le quatrième chapitre du cours, et concerne la programmation CUDA. Les mêmes commentaires que ceux des derniers TP s’appliquent ici aussi.**
>
> **En imagerie numérique, l'égalisation d'histogramme est une méthode d'ajustement du contraste d'une image donnée (cf. http://en.wikipedia.org/wiki/Histogram_equalization). Pour une image en niveaux de gris, l'idée est de calculer un histogramme comptant l'utilisation de chaque niveau de gris, de calculer la fonction de répartition de cet histogramme, puis d'étaler les niveaux de gris utilisés.**
> 
> **Plus précisément, soit $\left\{x_i\right\}$ l'ensemble des pixels d'une image définie sur $L$ niveaux de gris. L'histogramme est un tableau comptant les occurrences de chaque niveau de gris noté $l$, pour $l\in\left[0\ldots L-1\right]$ : $$h\left(l\right)=\sum_{i=0}^{n-1}\delta\left(x_i-l\right),$$ où $n$ est le nombre de pixels de l'image, et $\delta$ est la fonction de Dirac telle que : $$\delta\left(\xi\right)=\left\{\begin{matrix}1\mathrm{\ si\ }\xi=0,\\0\mathrm{\ sinon.}\\\end{matrix}\right. $$**
>
> **La fonction de répartition $r$ est définie sur l'intervalle des niveaux de gris comme la somme des nombres d'occurrence des valeurs précédentes :
$$r\left(l\right)=\sum_{k=0}^{l}h\left(k\right).$$**
>
> **Eh oui, c’est une somme préfixe, donc un SCAN inclusif ! La transformation suivante permet « d’étaler » l'histogramme :
$$
T\left(x_i\right)=\frac{L-1}{L\times n}r\left(x_i\right).
$$**
>
> **Notez un point important pour $T$ : la division requière un calcul soit en virgule flottante, soit en l’effectuant en toute fin du calcul : d’abord le quotient $\left(L-1\right)\times r\left(x_i\right)$, puis la division.**
>
> **Cette méthode est étendue aux images couleurs en appliquant cette transformation sur la composante « intensité » (V) de la couleur exprimée dans le repère HSV : Hue (Teinte), Saturation et Value (cf. http://en.wikipedia.org/wiki/HSL_and_HSV). Avec des images 24 bits, la valeur s’exprime sur 1 octets ; donc L vaut 256 (et L-1=255).**
>
> **Nous allons jouer avec l’implantation de l’histogramme et le scan inclusif.**
>
> **<font color=pink>N'oubliez d'exécuter les quatre premières cellules de code afin d'installer l'extension CUDA et de vérifier son bon fonctionnement.</font>**

## <font color=green>Installation du sous-sytème</font>

In [ ]:
# vérifions l'installation du SDK Cuda ...
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# Installons l'extension CUDA (n'hésitez par à aller sur la page GitHub ...)
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git &> /dev/null
%load_ext nvcc_plugin
# Installons gdown pour charger fichier depuis Google Drive
!pip install --upgrade --no-cache-dir gdown &> /dev/null
# Installons g++-8
!sudo apt install g++-8 &> /dev/null
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-7 700 --slave /usr/bin/g++ g++ /usr/bin/g++-7
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-8 800 --slave /usr/bin/g++ g++ /usr/bin/g++-8
# importation Python pour charger/afficher des images
from google.colab.patches import cv2_imshow
import cv2
def afficher(file, width):
  img = cv2.imread(file)
  height = int(img.shape[0] * width / float(img.shape[1]))
  resized = cv2.resize(img, (width, height), interpolation = cv2.INTER_AREA) 
  cv2_imshow(resized)

---
# <font color=green>TP</font>
> L'installation s'est bien déroulée ? Parfait, maintenant au travail !
>
> En premier, il faut charger le TP8 depuis le drive Google ... Vous pouvez charger ce fichier (*i.e.* le premier, le second contient des images) sur votre ordinateur pour étudiez les interfaces, bien que la plupart soient dans le cours ...


In [ ]:
# Chargeons le TP8
!gdown https://drive.google.com/uc?id=1LdDZmQEh0rH7kaQmDTsmpRbMafP8AXan
!gdown https://drive.google.com/uc?id=1bn8w-lhQNG7K1Ji4NVBRKHb0COoRmJSd
!unzip -oqq TP8.zip
!unzip -oqq Images-TP8.zip 
!ls Images


>
> Le code du TP est dans le répertoire TP8. Vous pouvez le vérifier dans une cellule en tapant " !ls TP8" par exemple ...
>
> Nous démarrons avec l'exercice 1. 
---
## <font color=green>Exercice 1</font>
>
> **Implémentez la fonction `rgb2hsv` qui, pour chaque pixel de l'image, calcule sa valeur dans l'espace HSV en utilisant la fonction `RGB2HSV`, et répartit le résultat dans trois tableaux différents. Notez qu'il s'agit d'une forme de SCATTER. Ce type de répartition en trois tableaux vise à optimiser le débit mémoire d'un kernel CUDA (encore la coalescence).**
>
> **Implémentez la transformation inverse (`hsv2rgb`), de HSV vers RGB, en utilisant la fonction `HSV2RGB`.**
**
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante. 
>
> Pour sauvegarder, n'oubliez pas de terminer par "Shift-Entrée" ... 
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**

In [ ]:
%%cuda --name ../TP8/student/exo1/student.cu 
#include <iostream>
#include <exo1/student.h>
#include <OPP_cuda.cuh>

#ifndef WIN32
# include <cfloat>
#endif

namespace 
{
	__device__
	float3 RGB2HSV( const uchar3 inRGB ) {
		const float R = float( inRGB.x ) / 256.f;
		const float G = float( inRGB.y ) / 256.f;
		const float B = float( inRGB.z ) / 256.f;

		const float min		= fminf( R, fminf( G, B ) );
		const float max		= fmaxf( R, fmaxf( G, B ) );
		const float delta	= max - min;

		// H
		float H;
		if		( delta < FLT_EPSILON )  
			H = 0.f;
		else if	( max == R )	
			H = 60.f * ( G - B ) / ( delta + FLT_EPSILON )+ 360.f;
		else if ( max == G )	
			H = 60.f * ( B - R ) / ( delta + FLT_EPSILON ) + 120.f;
		else					
			H = 60.f * ( R - G ) / ( delta + FLT_EPSILON ) + 240.f;
		while	( H >= 360.f )	
			H -= 360.f ;

		// S
		const float S = max < FLT_EPSILON ? 0.f : 1.f - min / max;

		// V
		const float V = max;

		return make_float3( H, S, V );
	}

	__device__
	uchar3 HSV2RGB( const float H, const float S, const float V ) {
		const float	d	= H / 60.f;
		const int	hi	= int(d) % 6;
		const float f	= d - float(hi);

		const float l   = V * ( 1.f - S );
		const float m	= V * ( 1.f - f * S );
		const float n	= V * ( 1.f - ( 1.f - f ) * S );

		float R, G, B;

		if		( hi == 0 ) 
			{ R = V; G = n;	B = l; }
		else if ( hi == 1 ) 
			{ R = m; G = V;	B = l; }
		else if ( hi == 2 ) 
			{ R = l; G = V;	B = n; }
		else if ( hi == 3 ) 
			{ R = l; G = m;	B = V; }
		else if ( hi == 4 ) 
			{ R = n; G = l;	B = V; }
		else				
			{ R = V; G = l;	B = m; }
			
		return make_uchar3( R * 256.f, G * 256.f, B * 256.f );
	}
}

bool StudentWorkImpl::isImplemented() const {
	return true;
}

void StudentWorkImpl::run_RGB2HSV(
	OPP::CUDA::DeviceBuffer<uchar3>& dev_source,
	OPP::CUDA::DeviceBuffer<float>& dev_Hue,
	OPP::CUDA::DeviceBuffer<float>& dev_Saturation,
	OPP::CUDA::DeviceBuffer<float>& dev_Value,
	const unsigned width,
	const unsigned height
) {
	// TODO
}

void StudentWorkImpl::run_HSV2RGB(
	OPP::CUDA::DeviceBuffer<float>& dev_Hue,
	OPP::CUDA::DeviceBuffer<float>& dev_Saturation,
	OPP::CUDA::DeviceBuffer<float>& dev_Value,
	OPP::CUDA::DeviceBuffer<uchar3>& dev_result,
	const unsigned width,
	const unsigned height
) {
	// TODO
}

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP8 ; sh ./build.sh exo1

> ### <font color=green>Exécution</font>
> Exécutez la cellule suivante pour exécuter le code ...

In [ ]:
# launch student work
!./TP8/linux/exo1 -i=./Images/Nuit.ppm
# display input image
print("\nInput image is:")
afficher(file="./Images/Nuit.ppm", width=600)
# display result
print("\nYour result is:")
afficher(file="Images/Nuit_RGB2HSV2RGB.ppm", width = 600) 

## <font color=green>Exercice 2</font>

> **Calculez l’histogramme des valeurs.**
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante. 
>
> Pour sauvegarder, n'oubliez pas de terminer par "Shift-Entrée" ... 
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**


In [ ]:
%%cuda --name ../TP8/student/exo2/student.cu
#include <iostream>
#include <exo2/student.h>
#include <OPP_cuda.cuh>

namespace 
{
}

bool StudentWorkImpl::isImplemented() const {
	return true;
}

void StudentWorkImpl::run_Histogram(
	OPP::CUDA::DeviceBuffer<float>& dev_value,
	OPP::CUDA::DeviceBuffer<unsigned>& dev_histogram,
	const unsigned width,
	const unsigned height
) {
	// TODO
}

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cat TP8/utils/OPP/OPP_cuda_histogram.cuh
!cd TP8 ; sh ./build.sh exo2

> ### <font color=green>Exécution</font>
> Exécutez les cellules suivantes pour exécuter le code (avec les images pré-chargées) ...

In [ ]:
# launch student work 
!./TP8/linux/exo2 -i=./Images/Nuit.ppm

In [ ]:
# launch student work
!./TP8/linux/exo2 -i=./Images/Roy_Lichtenstein_Drowning_Girl.ppm

In [ ]:
# launch student work
!./TP8/linux/exo2 -i=./Images/The_Nightwatch_by_Rembrandt.ppm

## <font color=green>Exercice 3</font>

> **Calculez la fonction de répartition.**
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante. 
>
> Pour sauvegarder, n'oubliez pas de terminer par "Shift-Entrée" ... 
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**


In [ ]:
%%cuda --name ../TP8/student/exo3/student.cu
#include <iostream>
#include <exo3/student.h>
#include <OPP_cuda.cuh>

namespace 
{
}

bool StudentWorkImpl::isImplemented() const {
	return true;
}

void StudentWorkImpl::run_Repartition(
	OPP::CUDA::DeviceBuffer<unsigned>& dev_histogram,
	OPP::CUDA::DeviceBuffer<unsigned>& dev_repartition
) {
	// TODO
}

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP8 ; sh ./build.sh exo3

> ### <font color=green>Exécution</font>
> Exécutez les cellules suivantes pour exécuter le code (avec les images pré-chargées) ...

In [ ]:
# launch student work
!./TP8/linux/exo3 -i=./Images/Nuit.ppm

In [ ]:
# launch student work
!./TP8/linux/exo3 -i=./Images/Roy_Lichtenstein_Drowning_Girl.ppm

In [ ]:
# launch student work
!./TP8/linux/exo3 -i=./Images/The_Nightwatch_by_Rembrandt.ppm

## <font color=green>Exercice 4</font>

> **Calculez la transformation finale.**
>
> **Admirez.**
>
> **Comme toujours, votre rapport doit discuter les performances en fonctions des nombres de threads et de leur répartition.**
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante. 
>
> Pour sauvegarder, n'oubliez pas de terminer par "Shift-Entrée" ... 
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**

In [ ]:
%%cuda --name ../TP8/student/exo4/student.cu
#include <iostream>
#include <exo4/student.h>
#include <OPP_cuda.cuh>

namespace 
{
}

bool StudentWorkImpl::isImplemented() const {
	return true;
}

void StudentWorkImpl::run_Transformation(
	OPP::CUDA::DeviceBuffer<float>& dev_Value,
	OPP::CUDA::DeviceBuffer<unsigned>& dev_repartition,
	OPP::CUDA::DeviceBuffer<float>& dev_transformation // or "transformed"
) {
	// TODO
}

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP8 ; sh ./build.sh exo4

> ### <font color=green>Exécution</font>
> Exécutez les cellules suivantes pour exécuter le code (avec les images pré-chargées) ...

In [ ]:
# launch student work 
!./TP8/linux/exo4 -i=./Images/Hopper.railroad.ppm
# display reference
afficher(file="Images/Hopper.railroad.ppm", width = 600) 
# display result
afficher(file="Images/Hopper.railroad_equalized.ppm", width = 600) 
# width = mettez une largeur en fonction de votre bande passante Internet 

In [ ]:
# launch student work 
!./TP8/linux/exo4 -i=./Images/Nuit.ppm
# display reference
afficher(file="Images/Nuit.ppm", width = 600) 
# display result
afficher(file="Images/Nuit_equalized.ppm", width = 600) 
# width = mettez une largeur en fonction de votre bande passante Internet 

In [ ]:
# launch student work 
!./TP8/linux/exo4 -i=./Images/Paris.ppm
# display reference
afficher(file="Images/Paris.ppm", width = 600) 
# display result
afficher(file="Images/Paris_equalized.ppm", width = 600) 
# width = mettez une largeur en fonction de votre bande passante Internet 

In [ ]:
# launch student work 
!./TP8/linux/exo4 -i=./Images/Roy_Lichtenstein_Drowning_Girl.ppm
# display reference
afficher(file="Images/Roy_Lichtenstein_Drowning_Girl.ppm", width = 600) 
# display result
afficher(file="Images/Roy_Lichtenstein_Drowning_Girl_equalized.ppm", width = 600) 
# width = mettez une largeur en fonction de votre bande passante Internet 

In [ ]:
# launch student work 
!./TP8/linux/exo4 -i=./Images/SunFlowers.ppm
# display reference
afficher(file="Images/SunFlowers.ppm", width = 600) 
# display result
afficher(file="Images/SunFlowers_equalized.ppm", width = 600) 
# width = mettez une largeur en fonction de votre bande passante Internet 

In [ ]:
# launch student work 
!./TP8/linux/exo4 -i=./Images/The_Nightwatch_by_Rembrandt.ppm
# display reference
afficher(file="Images/The_Nightwatch_by_Rembrandt.ppm", width = 600) 
# display result
afficher(file="Images/The_Nightwatch_by_Rembrandt_equalized.ppm", width = 600) 
# width = mettez une largeur en fonction de votre bande passante Internet 

In [ ]:
# launch student work 
!./TP8/linux/exo4 -i=./Images/Unequalized_Hawkes_Bay_NZ.ppm
# display reference
afficher(file="Images/Unequalized_Hawkes_Bay_NZ.ppm", width = 600) 
# display result
afficher(file="Images/Unequalized_Hawkes_Bay_NZ_equalized.ppm", width = 600) 
# width = mettez une largeur en fonction de votre bande passante Internet 

# <font color=green>That's all, folks!</font>